# Research Question 1

## Does the style differ between different authors and projects?

a) Authors: See evaluation below

b) Projects: Not (yet) implemented [Dataloaders and Pairs are built up using authors only]

In [1]:
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as L

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Style_Model from trained model

# Maybe use state_dict later: https://wandb.ai/wandb/common-ml-errors/reports/How-to-Save-and-Load-Models-in-PyTorch--VmlldzozMjg0MTE

import sys
sys.path.append('..')

from util.style_model import StyleModel

model = StyleModel()
model.load_state_dict(torch.load('../model/Style_Model.pt'))

<All keys matched successfully>

In [3]:
# load testing data
test_data = pd.read_pickle('../data/04c_Test_Set.pkl')

### Calculate Embedding Centroid for each author

In [4]:
authors_centroids = {}
counter = 0

for group in test_data.groupby('author_email'):
    if counter == 2:
        break
    author_embeddings = np.array([model(message).squeeze().detach().numpy() for message in group[1]['message']])
    authors_centroids[group[0]] = np.mean(author_embeddings, axis = 0)
    counter += 1

### Calculate Cosine Distance between Author Centroids

In [12]:
from sklearn.metrics.pairwise import euclidean_distances

centroids_array = [value for value in authors_centroids.values()]

euclidean_distances(centroids_array, centroids_array)

array([[0.0000000e+00, 1.6516577e-01],
       [1.6516577e-01, 2.1073424e-08]], dtype=float32)

### Half-Automated Evaluation of Style by K-Means Summary

Check K-Means Summary for Number of Authors and Projects per Cluster. If the number is low, it indicates that styles can be assigned to the different authors.